In [10]:
import numpy as np
from PySDM import Formulae
from PySDM.physics import si
from PySDM.initialisation.spectra import Lognormal
from PySDM.products import (
    ParcelDisplacement,
    AmbientTemperature,
    AmbientDryAirDensity,
    AmbientRelativeHumidity,
    ParticleSizeSpectrumPerVolume,
    ParticleVolumeVersusRadiusLogarithmSpectrum,
)

from PySDM_examples.Pyrcel import Settings, Simulation
import sys
import os

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
exporters_path = os.path.join(notebook_dir, "../../../PySDM/exporters/")
print(exporters_path)
sys.path.append(exporters_path)

from parcel_vtk_exporter import VTKExporterPyrcel

c:\Users\strza\Desktop\PySDM\examples\PySDM_examples\Pyrcel\../../../PySDM/exporters/


In [11]:
settings = Settings(
    dz=10 * si.m,
    n_sd_per_mode=(25, 25),
    aerosol_modes_by_kappa={
        0.54: Lognormal(norm_factor=850 / si.cm**3, m_mode=15 * si.nm, s_geom=1.6),
        1.2: Lognormal(norm_factor=10 / si.cm**3, m_mode=850 * si.nm, s_geom=1.2),
    },
    vertical_velocity=1.0 * si.m / si.s,
    initial_pressure=775 * si.mbar,
    initial_temperature=274 * si.K,
    initial_relative_humidity=0.90,
    displacement=1000 * si.m,
    formulae=Formulae(constants={"MAC": 0.3}),
)

dry_radius_bin_edges = np.logspace(
    np.log10(1e-3 * si.um), np.log10(5e0 * si.um), 33, endpoint=False
)

simulation = Simulation(
    settings,
    products=(
        ParcelDisplacement(name="z"),
        AmbientRelativeHumidity(name="S_max_percent", unit="%", var="RH"),
        AmbientTemperature(name="T"),
        ParticleSizeSpectrumPerVolume(
            name="dry:dN/dR", radius_bins_edges=dry_radius_bin_edges, dry=True
        ),
        ParticleVolumeVersusRadiusLogarithmSpectrum(
            name="dry:dV/dlnR", radius_bins_edges=dry_radius_bin_edges, dry=True
        ),
        AmbientDryAirDensity(),
    ),
)

In [12]:
output = simulation.run()

In [13]:
e = VTKExporterPyrcel(n_sd=simulation.particulator.n_sd, output=output, mass_of_dry_air=simulation.particulator.environment.mass_of_dry_air)
for step in settings.output_steps:
    e.export_products(step, simulation)
    e.export_attributes(step, simulation)
e.write_pvd()